In [1]:

import os, zipfile, numpy as np, pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [10]:
DTA_PATH = "C:/Users/wayne/Dal/CSCI_6055_R/gss_green_eds/GSS2018.dta"
MODEL_RESULTS_PATH = "C:/Users/wayne/Dal/CSCI_6055_R/gss_green_eds/model_results_fulladjustment_updated_20240208.csv"
MORE_MODEL_RESULTS_PATH = "C:/Users/wayne/Dal/CSCI_6055_R/gss_green_eds/more_model_results_fulladjustment_updated20240208.csv"

In [11]:
# Load into separate DataFrames
df = pd.read_stata(DTA_PATH, convert_categoricals=False)
df_main = pd.read_csv(MODEL_RESULTS_PATH)
df_more = pd.read_csv(MORE_MODEL_RESULTS_PATH)

In [6]:
df_main

,Unnamed: 0,naturevar,subgroup,beta,se,deviance,RR,LL,UL
0,m1,nattimeok,main,0.021647,0.011462,528.747540,1.02,1.00,1.05
1,m2,nattimeok,men,0.008047,0.018703,520.442683,1.01,0.97,1.05
2,m3,nattimeok,women,0.034424,0.016361,520.442683,1.04,1.00,1.07
3,m3a,nattimeok,missing gender,-0.027717,0.049697,520.442683,0.97,0.88,1.07
4,m1lrt,nattimeok,gender adj for LRT,0.021656,0.011412,521.352008,1.02,1.00,1.05
5,m4,nattimeok,white,0.028861,0.014305,514.539080,1.03,1.00,1.06
6,m5,nattimeok,nonwhite,0.015655,0.028494,514.539080,1.02,0.96,1.07
7,m2lrt,nattimeok,race adj for LRT,0.025295,0.011597,514.659300,1.03,1.00,1.05
8,m6,natsat,main,0.063397,0.021444,383.712329,1.07,1.02,1.11
9,m7,natsat,men,0.021943,0.037783,375.783753,1.02,0.95,1.10


In [7]:
df_more

,Unnamed: 0,naturevar,subgroup,beta,se,deviance,RR,LL,UL
0,m11,nat_timeoncewk,main,0.002381,0.018529,529.285358,1.00,0.97,1.04
1,m12,nat_timeoncewk,men,0.010117,0.023432,523.807637,1.01,0.96,1.06
2,m13,nat_timeoncewk,women,0.014008,0.025320,523.807637,1.01,0.97,1.07
3,m13a,nat_timeoncewk,missinggender,-0.092928,0.066735,523.807637,0.91,0.80,1.04
4,m5lrt,nat_timeoncewk,gender adj for lrt,0.003407,0.018569,526.290841,1.00,0.97,1.04
5,m14,nat_timeoncewk,white,-0.012505,0.024610,519.248840,0.99,0.94,1.04
6,m15,nat_timeoncewk,nonwhite,0.038073,0.025222,519.248840,1.04,0.99,1.09
7,m6lrt,nat_timeoncewk,race adj for lrt,0.004426,0.018536,520.977644,1.00,0.97,1.04


In [12]:
df

,abany,abdefect,abfelegl,abhelp1,abhelp2,abhelp3,abhelp4,abhlth,abinspay,abmedgov1,...,xmarsex,xmarsex1,xmovie,xnorcsiz,year,yearsjob,yearsusa,yearval,yousup,zodiac
0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,NaN,6,2018,1.0,NaN,NaN,45.0,6.0
1,1.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,NaN,...,1.0,NaN,2.0,6,2018,NaN,NaN,NaN,NaN,11.0
2,NaN,NaN,NaN,1.0,2.0,1.0,1.0,NaN,2.0,1.0,...,NaN,1.0,2.0,6,2018,15.0,NaN,NaN,3.0,1.0
3,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,2.0,6,2018,25.0,NaN,NaN,10.0,1.0
4,2.0,1.0,NaN,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,1.0,NaN,2.0,6,2018,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,1.0,1.0,NaN,10,2018,2.0,NaN,NaN,NaN,5.0
2344,2.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,10,2018,2.0,NaN,NaN,1.0,9.0
2345,2.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,1.0,NaN,...,1.0,NaN,2.0,10,2018,NaN,NaN,NaN,NaN,NaN
2346,NaN,NaN,NaN,2.0,2.0,2.0,1.0,NaN,1.0,1.0,...,NaN,1.0,2.0,10,2018,NaN,NaN,NaN,NaN,2.0


In [13]:
df.columns.tolist()


['abany',
 'abdefect',
 'abfelegl',
 'abhelp1',
 'abhelp2',
 'abhelp3',
 'abhelp4',
 'abhlth',
 'abinspay',
 'abmedgov1',
 'abmedgov2',
 'abmelegl',
 'abmoral',
 'abnomore',
 'abpoor',
 'abpoorw',
 'abrape',
 'absingle',
 'abstate1',
 'abstate2',
 'acqntsex',
 'actssoc',
 'adminconsent',
 'adults',
 'advfront',
 'affrmact',
 'afraidof',
 'afterlif',
 'age',
 'aged',
 'agekdbrn',
 'ancestrs',
 'arthrtis',
 'astrolgy',
 'astrosci',
 'atheists',
 'attend',
 'attend12',
 'attendma',
 'attendpa',
 'away1',
 'away11',
 'away2',
 'away3',
 'away4',
 'away5',
 'away6',
 'away7',
 'babies',
 'backpain',
 'ballot',
 'balneg',
 'balpos',
 'befair',
 'betrlang',
 'bible',
 'bigbang',
 'bigbang1',
 'bigbang2',
 'bird',
 'birdb4',
 'born',
 'boyorgrl',
 'breakdwn',
 'buddhsts',
 'buyesop',
 'buyvalue',
 'cantrust',
 'cappun',
 'cat',
 'catb4',
 'charactr',
 'chemgen',
 'childs',
 'chldidel',
 'christns',
 'churhpow',
 'class',
 'clergvte',
 'closeto1',
 'closeto2',
 'closeto3',
 'closeto4',
 'closet

In [16]:
df['EDS_raw'] = df[['disrspct', 'poorserv', 'notsmart', 'afraidof', 'threaten']].sum(axis=1)
df['EDS_z'] = (df['EDS_raw'] - df['EDS_raw'].mean()) / df['EDS_raw'].std()

C:\Users\wayne\AppData\Local\Temp\ipykernel_4804\3259495321.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['EDS_raw'] = df[['disrspct', 'poorserv', 'notsmart', 'afraidof', 'threaten']].sum(axis=1)
C:\Users\wayne\AppData\Local\Temp\ipykernel_4804\3259495321.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['EDS_z'] = (df['EDS_raw'] - df['EDS_raw'].mean()) / df['EDS_raw'].std()
